In [2]:
import numpy as np
import pandas as pd
import os
from typing import List, Dict, Tuple
from tqdm import tqdm


### Detektor Evaluation
- Finde Repeatability und Accuracy für die verschiedenen Models

In [ ]:
def find_nearest_neighbour_without_duplicates(
    values:np.array) -> Tuple[np.array, np.array]:
    """Find Nearest Neighbour without two keypoints form image j are appointed to the same keypoint in image i.
    1) If two keypoints kp_j1 and kp_j2 are appointed to the same keypoint kp_i1, then find the better matching one and keep it.
    2) The other match is discarded and the next best match for that
    keypoint is taken. 
    3) Check for duplicate matches and go back to 1) if the situation arises.
    """

    # For each kp_j find the best matching order (ascending distance) for all
    # of kp_i.
    l2 = values.copy()
    min_idx = l2.argsort(axis=1)

    # Prevents infinte loop, if more kp_j exists than kp_i
    _MAX_ITERATION_COUNT = l2.shape[1]
    _iteration_count = 0

    # Assume, duplicates exists to test at least once.
    has_duplicates = True

    while(has_duplicates and _iteration_count < _MAX_ITERATION_COUNT):
        has_duplicates = False
        _iteration_count += 1

        # Get number of unique ids of the best matches kp_i and count, how
        # often kp_1 has been assigned.
        _dup_idx, _counts = np.unique(min_idx[:, 0], return_counts=True)
        dup_and_count = np.vstack([_dup_idx,_counts]).T

        # Check for actual duplicate assignments. If ids from kp_i has been 
        # assigned multiple times, the number of unique ids will be smaller
        # than the number of kp_j (= rows). 
        if dup_and_count.shape[0] != min_idx.shape[0]:
            has_duplicates = True

            # Count how often an Keypoint kp_i has been matched more than once
            # and get the first kp_i id. This kp_x will be fixed first.
            gt_1 = dup_and_count[:, 1] > 1
            dup_id = dup_and_count[gt_1][0][0]

            # Find all matches where kp_x is the best match. Those rows are 
            # candidates to be fixed.
            candidates = min_idx[:, 0] == dup_id

            # Of all those candidates, find the match with the smallest value.
            # This is the best match for that kp_x.
            # All other rows with kp_x as best match are going to be rotated.
            l2_args = l2[:, 0].argsort()
            cand_args = candidates[l2_args]
            best_candidates_id = l2[:, 0].argsort()[cand_args][0]

            # Mark the best match as "False" to not rotate that row.
            candidates[best_candidates_id] = False

            # Then circle to all the other matches and rotate the row to get 
            # the next best match to be their best match.
            for id_c, val_c in enumerate(candidates):
                if val_c:
                min_idx[id_c] = np.roll(min_idx[id_c], -1)
                l2[id_c] = np.roll(l2[id_c], -1)

        # If the while loop ends, all kp_i have been assigned once.
        # Return the best match ids of kp_i and their respective distance values.
        return l2[:, 0], min_idx[:, 0]

In [8]:


def get_set_names(data_dir:str, sort_output:bool=True) -> List[str]:
    set_names = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x))]

    if sort_output:
        set_names = sorted(set_names)
    
    return set_names

def get_file_names_in_set(path_set:str, file_sheme:str, sort_output:bool=True) -> List[str]:
    file_names = [x for x in os.listdir(path_set) if os.path.isfile(os.path.join(path_set, x))]
 
    # get the correct files with fitting file scheme.
    file_names = [x for x in file_names if file_scheme in x]

    if sort_output:
        file_names = sorted(file_names)

    return file_names

def evaluate_detector(
    detector_name:str,
    collection_name:str,
    path_collection:str,
    set_names:List[str],
    file_scheme:str,
    keypoint_thresholds:List[int],
    dist_error_thresholds:List[float],
    error_vals_per_set:Dict,
    column_names:List[str],
    fast_eval:bool=False) -> pd.DataFrame:

    # Create output dataframe.
    df = pd.DataFrame(columns=column_names)

    for set_name in set_names:
        path_set = os.path.join(path_collection, set_name, 'keypoints', detector_name)
        file_names = get_file_names_in_set(path_set, file_scheme)
        num_files = 10 if fast_eval else len(file_names)
        for i in tqdm(range(num_files)):
            path_f1 = os.path.join(path_set, file_names[i])
            f1 = pd.read_csv(path_f1, sep=',', header=None, usecols=[0, 1], comment='#').values.astype('float32')
            num_kpts_i = f1.shape[0]

            for j in range(i+1,num_files,1):
                path_f2 = os.path.join(path_set, file_names[j])
                f2 = pd.read_csv(path_f2, sep=',', header=None, usecols=[0, 1], comment='#').values.astype('float32')
                num_kpts_j = f2.shape[0]


                for kp_thresh in keypoint_thresholds:
                    _f1 = f1[:kp_thresh]
                    _f2 = f2[:kp_thresh]

                    # Number of maximal possible matches for first t keypoints.
                    max_num_matches = np.min([len(_f1), len(_f2)])

                    # Each row k contains the differences f2_k - f1, for all
                    # f1_l in f1.
                    # [[f2_0 - f1_0, f2_0 - f1_1, ..., f2_0 - f1_l],
                    #  [...]
                    #  [f2_m - f1_0, f2_m - f1_1, ..., f2_m - f1_l]]
                    _d = np.linalg.norm(_f2 - _f1[:, np.newaxis], axis=2)
                    
                    # Get the index of the lowest squared difference for each row
                    sorted_idx = _d.argsort(axis=1)
                    _nn = sorted_idx[:, 0] # indices of the nearest neighbour kpts

                    # Get the corresponding d value
                    _d = _d[:, _nn][:, 0]
                    
                    for dist_percentage in dist_error_thresholds:
                        # Compute the distance threshold as L2 Norm value
                        dist_thresh = error_vals_per_set[set_name] * dist_percentage
                        dist_thresh = np.linalg.norm(dist_thresh)
                        
                        # Remove all entries, that violate dist_thresh.
                        nn = _nn[_d <= dist_thresh]
                        d = _d[_d <= dist_thresh]
                        
                        # Find duplicates
                        _, u_idx = np.unique(nn, return_index=True)

                        # Remove duplicates
                        d = d[u_idx]

                        # mean distance of all hit
                        min_dist = np.min(d) if len(d) else -1
                        max_dist = np.max(d) if len(d) else -1
                        mean_dist = np.mean(d) if len(d) else -1
                        std_dist = np.std(d) if len(d) else -1

                        # repeatability: ratio of matches and number of maximal possible
                        # matches:
                        num_matches = len(d)
                        repeatability = 0 if max_num_matches == 0 else num_matches / max_num_matches
                        
                        # The accuracy of a match is 1 - (l2.distance / dist_tresh)
                        # And the accuracy for the image pair is the mean of all accuracie values
                        #acc_values = 1.0 - (d / dist_thresh)
                        #acc_values = 1.0 - np.divide(d, dist_thresh, out=np.zeros_like(d), where=d > 0)
                        acc_values = np.divide(1.0, np.sqrt(d + 1.0), out=np.zeros_like(d), where=d>=0)
                        accuracy = np.mean(acc_values)
                        
                        # Append new row to dataframe.
                        df = df.append({
                            'collection_name': collection_name,
                            'set_name': set_name,
                            'detector_name': detector_name,
                            'image_i': file_names[i],
                            'image_j': file_names[j],
                            'num_kpts_i': num_kpts_i,
                            'num_kpts_j': num_kpts_j,
                            'keypoint_threshold': kp_thresh,
                            'dist_threshold': dist_percentage,
                            'max_num_matches': max_num_matches,
                            'num_matches': num_matches,
                            'mean_dist': mean_dist,
                            'std_dist': std_dist,
                            'min_dist': min_dist,
                            'max_dist': max_dist,
                            'min_dist': min_dist,
                            'max_dist': max_dist,
                            'repeatability': repeatability,
                            'accuracy': accuracy
                            }, ignore_index=True)
    return df

def save_output_for_detector(
    path_output:str,
    detector_name:str,
    collection_name:str,
    df:pd.DataFrame) -> None:

    fout_name = '{}_{}.csv'.format(detector_name, collection_name)
    if not os.path.exists(path_output):
        os.makedirs(path_output, exist_ok=True)

    df.to_csv(os.path.join(path_output, fout_name), 
              index=False, 
              encoding='utf-8')


#################################
### PARAMS
#################################

root_dir = '/home/mizzade/Workspace/diplom/code' # Adjust this accordingly
data_dir = 'outputs'
output_dir = 'output_evaluation'
path_output = os.path.join(root_dir, output_dir, 'detectors')

collection_name = 'webcam'
path_collection = os.path.join(root_dir, data_dir, collection_name)

file_scheme = '_10000.csv'
detector_names = ['sift', 'lift', 'tcovdet' , 'tilde', 'superpoint']
detector_name = 'sift'

set_names = get_set_names(path_collection, sort_output=True)
keypoint_thresholds = [1000, 5000, 10000]
#dist_error_thresholds = [1, 5, 10, 15, 20, 25, 30, 35, 40]
dist_error_thresholds = [14]
#dist_error_thresholds = np.linspace(1,200, 200)
fast_eval = False


# Width, Height
# Number of pixels for width and height that make 1% of the corresponding
# dimension for the images in the corresponding set.
error_vals_per_set = {
    'chamonix': np.array([704, 547]) / 100.0,
    'courbevoie': np.array([640, 471]) / 100.0,
    'frankfurt': np.array([1024, 627]) / 100.0,
    'mexico': np.array([640, 418]) / 100.0,
    'panorama': np.array([2469, 205]) / 100.0,
    'stlouis': np.array([800, 450]) / 100.0,
    'v_xxl': np.array([1000, 1000]) / 100.0
}

# 'collection_name':str           Name of the collection.
# 'set_name':str                  Name of the set.         
# 'detector_name':str             Name of the detector.
# 'image_i':str                   Name of the first (left) image.
# 'image_j':str                   Name of the second (right) image.
# 'num_kpts_i':int                Number of keypoints found in first image.
# 'num_kpts_j':int                Number of keypoints found in the second image.
# 'keypoint_threshold':int        Number of keypoints to use. [1000, 5000, 10000].
# 'dist_percentage':float         Maximal match distance in percentage to count 
#                                 as match.Relative to image dimensions. [1, 5, 10]
# 'max_num_matches':int           Maximal number of possible matches under current 
#                                 conditions.
# 'num_matches':int               Actual number of matches.
# 'mean_dist':float               Average L2 distance between two matches.
# 'std_dist':float                Standard deviation of the l2 distance between 
#                                 two matches.
# 'min_dist':float                Smallest L2 distance between two matches.
# 'max_dist':float                Largest L2 distance between two matches.
# 'repeatability':float
column_names = ['collection_name','set_name', 'detector_name', 
                'image_i', 'image_j', 'num_kpts_i', 'num_kpts_j', 
                'keypoint_threshold', 'dist_threshold', 
                'max_num_matches', 'num_matches', 'mean_dist', 
                'std_dist', 'min_dist', 'max_dist', 'repeatability', 'accuracy']

# Test
# detector_names = ['sift']
# collection_name = 'example'
# set_names = ['v_xxl']
# path_collection = os.path.join(root_dir, data_dir, collection_name)
# error_vals_per_set['v_xxl'] = np.array([1000, 1000]) / 100.0

#################################
### MAIN
#################################

for detector_name in detector_names:
    print('Start evaluation of detector {}.'.format(detector_name))
    df = evaluate_detector(
     detector_name,
     collection_name,
     path_collection,
     set_names,
     file_scheme,
     keypoint_thresholds,
     dist_error_thresholds,
     error_vals_per_set,
     column_names,
     fast_eval=fast_eval)

    save_output_for_detector(
        path_output, 
        detector_name, 
        collection_name, 
        df)

    print('Evaluation of detector {} complete.'.format(detector_name))

  0%|          | 0/40 [00:00<?, ?it/s]

Start evaluation of detector sift.


  0%|          | 0/40 [00:00<?, ?it/s]

Evaluation of detector sift complete.
Start evaluation of detector lift.


  0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluation of detector tcovdet complete.
Start evaluation of detector tilde.


  0%|          | 0/40 [00:00<?, ?it/s]

Evaluation of detector tilde complete.
Start evaluation of detector superpoint.


100%|██████████| 40/40 [2:48:36<00:00, 252.91s/it]  


Evaluation of detector superpoint complete.
